<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@700&display=swap" rel="stylesheet">
</head>
<body>
    <center>
        <h2 style="font-weight: bolder; color: #488286; font-size: 180%; font-family: 'Times New Roman', sans-serif; text-shadow: 2px 2px 5px rgba(0, 0, 0, 0.1);">
            Regression-Driven Prediction of Medical Insurance Expenses & Premiums
        </h2>
    </center>
</body>
</html>

<div style="font-family: 'Times New Roman', serif; font-size: 20px; color: #000000; text-align: center; padding: 15px; border: 2px solid #ffffff; display: block; margin: auto;">
    Melissa Jalali Monfared
</div>

<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist">
 <p style="font-family:newtimeroman;color:#000000;font-size:120%;text-align:center;border-radius:40px 40px;">TABLE OF CONTENTS</p>   
    
* **[- | Introduction & Data Preprocessing](#0)**
    
* **[- | Data Visualization](#1)**
    
* **[- | Modeling](#2)**

<img src="https://www.gannett-cdn.com/authoring/authoring-images/2025/06/12/USAT/84166421007-usatgraphics-healthcarecoststopper.png?crop=3388,1906,x119,y5&width=2560" width="800" height="600">

<a id="0"></a>

<h1 style="
    background-color:#488286; 
    background-size: cover;
    background-position: center;
    font-family: 'Times New Roman', serif;
    font-size: 1.5em;
    color: white;
    text-align: center;
    padding: 15px;
    border-radius: 15px 50px;
    border: 1px solid black;
">
    Introduction & Data Preprocessing
</h1>

<div style="border: 2px solid #305252; padding: 15px; background-color: #ffffff;">

### <span style="color: #305252;">Medical Insurance Expenses & Premium Analysis</span>

Understanding the financial dynamics of healthcare is increasingly critical in designing fair and efficient insurance systems. This project presents a comprehensive data-driven analysis of medical insurance expenses and premiums, with a focus on uncovering the demographic and behavioral factors that influence healthcare costs.

The dataset used in this study includes detailed records of policyholders, capturing both demographic attributes (such as age, gender, BMI, number of children) and financial metrics (medical expenses and insurance premiums). By leveraging statistical modeling and machine learning techniques, I aim to build predictive models that estimate medical costs and premiums with high accuracy.

This exploration not only contributes to optimizing pricing strategies for insurance providers but also sheds light on equity in access and cost across different population segments. The project reflects my commitment to applying data science for impactful insights in the healthcare domain.

### <span style="color: #305252;">About Dataset</span>

| Column                 | Description                                                                 |
|------------------------|-----------------------------------------------------------------------------|
| **age**                | Age of the policyholder.                                                    |
| **gender**             | Gender of the policyholder (male/female).                                   |
| **bmi**                | Body Mass Index of the policyholder.                                        |
| **children**           | Number of children covered by the insurance policy.                         |
| **discount_eligibility** | Indicates whether the policyholder is eligible for a discount (yes/no).     |
| **region**             | Geographic region where the policyholder resides (e.g., southeast, northwest). |
| **expenses**           | Actual medical costs incurred by the policyholder. *(Target variable 1)*     |
| **premium**            | Insurance premium charged to the policyholder. *(Target variable 2)*         |

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from warnings import filterwarnings
filterwarnings("ignore")

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter

from ydata_profiling import ProfileReport

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet, BayesianRidge, HuberRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from matplotlib.colors import LinearSegmentedColormap

from sklearn.metrics import (
    accuracy_score, roc_auc_score, confusion_matrix, classification_report,
    mean_absolute_error, mean_squared_error, r2_score
)

from sklearn.model_selection import (
    train_test_split, cross_validate, GridSearchCV,
    KFold, cross_val_score
)

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

custom_palette = ["#B7D5D4", "#77878B", "#488286", "#305252"]
sns.set_palette(sns.color_palette(custom_palette))

In [ ]:
data = pd.read_csv('/kaggle/input/health-insurance-dataset/medical_insurance.csv')
data = pd.DataFrame(data)
data

In [ ]:
data.columns

In [ ]:
data.shape

- There are 8 columns & 1338 rows in this dataset.

In [ ]:
def check_df(data: object, head: object = 5) -> object:
    print("\nShape")
    print(data.shape)
    print("\nTypes")
    print(data.dtypes)
    print("\nNANs")
    print(data.isnull().sum())
    print("\nInfo")
    print(data.info())
check_df(data)

In [ ]:
print('Number of duplicated rows: ' , len(data[data.duplicated()]))

In [ ]:
data = data.drop_duplicates()
data = data.reset_index(drop=True)
print('Number of duplicated rows after cleaning:', data.duplicated().sum())

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
custom_colors = ["#B7D5D4", "#77878B", "#488286", "#305252"]
custom_cmap = LinearSegmentedColormap.from_list("custom_bone", custom_colors)

plt.figure(figsize=(22, 4))
sns.heatmap(
    (data.isna().sum()).to_frame(name='').T,
    cmap=custom_cmap,
    annot=True,
    fmt='0.0f'
).set_title('Count of Missing Values', fontsize=18)
plt.show()

In [ ]:
styled = data.describe().T.style.background_gradient(cmap=custom_cmap, axis=1)
styled

- we can see statistical information on the table above.

In [ ]:
# Finding unique data
data.apply(lambda x: len(x.unique()))

In [ ]:
unique = data.nunique().sort_values()
unique_values = data.apply(lambda x: x.unique())
pd.DataFrame({'Number of Unique Values': unique, 'Unique Values': unique_values})

In [ ]:
# Dropping non-numerical data
hm = data.drop(columns=['discount_eligibility', 'gender', 'region'])

In [ ]:
# Generate the profile report
profile = ProfileReport(data, 
                        title='Dataset Report', 
                        minimal=True, 
                        progress_bar=False, 
                        samples=None, 
                        correlations=None, 
                        interactions=None, 
                        explorative=True, 
                        notebook={'iframe': {'height': '600px'}}, 
                        html={'style': {'primary_color': '#C44536'}}, 
                        missing_diagrams={'heatmap': False, 'dendrogram': False})

# Display the report as an iframe in the notebook
profile.to_notebook_iframe()

<a id="1"></a>

<h1 style="
    background-color:#488286; 
    background-size: cover;
    background-position: center;
    font-family: 'Times New Roman', serif;
    font-size: 1.5em;
    color: white;
    text-align: center;
    padding: 15px;
    border-radius: 15px 50px;
    border: 1px solid black;
">
    Data Visualization
</h1>

In [ ]:
sns.set_palette(sns.color_palette(custom_palette))
columns = ['gender', 'children', 'discount_eligibility', 'region']
titles = ['Gender', 'Children', 'Discount Eligibility', 'Region']

def get_explode(values, threshold=10):
    total = sum(values)
    return [0.3 if (v / total) * 100 < threshold else 0 for v in values]

fig, ax = plt.subplots(figsize=(15, 12), nrows=2, ncols=2)

for i, column in enumerate(columns):
    row = i // 2
    col = i % 2
    values = data[column].value_counts()
    explode = get_explode(values)
    values.plot.pie(
        autopct='%1.1f%%',
        ax=ax[row, col],
        startangle=90,
        textprops={'color': 'black', 'fontsize': 10},
        pctdistance=0.7,
        explode=explode,
        colors=custom_palette
    )
    ax[row, col].set_title(titles[i], fontsize=14, loc='left')
    ax[row, col].set_ylabel('')

plt.tight_layout(pad=2.0)
plt.show()

In [ ]:
custom_palette = ["#B7D5D4", "#77878B", "#488286", "#305252"]

columns = ['gender', 'children', 'discount_eligibility', 'region']

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

for i, col_name in enumerate(columns):
    row = i // 2
    col = i % 2
    ax = sns.countplot(ax=axes[row, col], x=data[col_name], palette=custom_palette)
    total = len(data[col_name])
    for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_height() / total)
        x = p.get_x() + p.get_width() / 2
        y = p.get_height()
        ax.annotate(percentage, (x, y), ha='center', va='bottom', fontsize=10)
    axes[row, col].set_title(col_name.title(), fontsize=14)
    axes[row, col].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
styled_corr = hm.corr(numeric_only=True).T.style.background_gradient(cmap=custom_cmap, axis=1)
styled_corr

In [ ]:
plt.figure(figsize=(20, 12))
sns.heatmap(
    hm.corr(numeric_only=True),
    cmap=custom_cmap,
    annot=True,
    linewidths=0.6,
    cbar=False)

plt.xticks(rotation=60, size=10)
plt.yticks(size=10)
plt.title('Analysis of Correlations', size=20)
plt.tight_layout()
plt.show()

In [ ]:
corr = hm.corr(numeric_only=True)
f, ax = plt.subplots(figsize=(15, 5))
mask = np.triu(np.ones_like(corr, dtype=bool))
cut_off = 0.25
extreme_1 = 0.5
extreme_2 = 0.75
extreme_3 = 0.9
mask |= np.abs(corr) < cut_off
corr = corr[~mask]
remove_empty_rows_and_cols = True
if remove_empty_rows_and_cols:
    wanted_cols = np.flatnonzero(np.count_nonzero(~mask, axis=1))
    wanted_rows = np.flatnonzero(np.count_nonzero(~mask, axis=0))
    corr = corr.iloc[wanted_cols, wanted_rows]

annot = [[f"{val:.4f}"
          + ('' if abs(val) < extreme_1 else '\n*')
          + ('' if abs(val) < extreme_2 else '*')
          + ('' if abs(val) < extreme_3 else '*')
          for val in row] for row in corr.to_numpy()]
heatmap = sns.heatmap(corr, vmin=-1, vmax=1, annot=annot, fmt='', cmap=custom_cmap)
heatmap.set_title('Triangle Correlation Heatmap', fontdict={'fontsize': 12}, pad=16)
plt.show()

In [ ]:
plt.figure (figsize = (15 , 3) , dpi = 100)
heatmap = sns.heatmap (hm.corr(numeric_only=True)[['age']].sort_values (by = 'age', ascending = False), vmin = -1, vmax = 1, annot = True, cmap = custom_cmap)
heatmap.set_title ('Features Correlating with age', fontdict = {'fontsize':12} , pad = 18);

In [ ]:
plt.figure (figsize = (15 , 3) , dpi = 100)
heatmap = sns.heatmap (hm.corr(numeric_only=True)[['premium']].sort_values (by = 'premium', ascending = False), vmin = -1, vmax = 1, annot = True, cmap = custom_cmap)
heatmap.set_title ('Features Correlating with premium', fontdict = {'fontsize':12} , pad = 18);

In [ ]:
plt.figure (figsize = (15 , 3) , dpi = 100)
heatmap = sns.heatmap (hm.corr(numeric_only=True)[['expenses']].sort_values (by = 'expenses', ascending = False), vmin = -1, vmax = 1, annot = True, cmap = custom_cmap)
heatmap.set_title ('Features Correlating with expenses', fontdict = {'fontsize':12} , pad = 18);

In [ ]:
plt.figure (figsize = (15 , 3) , dpi = 100)
heatmap = sns.heatmap (hm.corr(numeric_only=True)[['bmi']].sort_values (by = 'bmi', ascending = False), vmin = -1, vmax = 1, annot = True, cmap = custom_cmap)
heatmap.set_title ('Features Correlating with bmi', fontdict = {'fontsize':12} , pad = 18);

In [ ]:
plt.figure (figsize = (15 , 3) , dpi = 100)
heatmap = sns.heatmap (hm.corr(numeric_only=True)[['children']].sort_values (by = 'children', ascending = False), vmin = -1, vmax = 1, annot = True, cmap = custom_cmap)
heatmap.set_title ('Features Correlating with children', fontdict = {'fontsize':12} , pad = 18);

In [ ]:
custom_palette2 = [
    "#B7D5D4",
    "#77878B",
    "#488286",
    "#305252",
    "#92B8A7",
    "#3B5A54"]

hm['children'] = hm['children'].astype(str)

sns.pairplot(
    data=hm,
    diag_kind='kde',
    hue='children',
    palette=custom_palette2,
    corner=True)
plt.show()

In [ ]:
ff = ['age', 'children', 'expenses', 'premium']

df_melted = data.melt(id_vars=['bmi', 'gender'], value_vars=ff, var_name='variable', value_name='value')

g = sns.relplot(
    data=df_melted,
    x='bmi',
    y='value',
    hue='gender',
    col='variable',
    kind='scatter',
    palette=custom_palette2,
    facet_kws={'sharey': False, 'sharex': True},
    height=5,
    aspect=1)

g.set_titles(col_template="{col_name}", size=16)
g.set_axis_labels("BMI", "")
for ax in g.axes.flatten():
    ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
ff = ['bmi', 'children', 'expenses', 'premium']

df_melted = data.melt(id_vars=['age', 'gender'], value_vars=ff, var_name='variable', value_name='value')

g = sns.relplot(
    data=df_melted,
    x='age',
    y='value',
    hue='gender',
    col='variable',
    kind='scatter',
    palette=custom_palette2,
    facet_kws={'sharey': False, 'sharex': True},
    height=5,
    aspect=1)

g.set_titles(col_template="{col_name}", size=16)
g.set_axis_labels("AGE", "")
for ax in g.axes.flatten():
    ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
ff = ['bmi', 'age', 'expenses', 'premium']

df_melted = data.melt(id_vars=['children', 'gender'], value_vars=ff, var_name='variable', value_name='value')

g = sns.relplot(
    data=df_melted,
    x='children',
    y='value',
    hue='gender',
    col='variable',
    kind='scatter',
    palette=custom_palette2,
    facet_kws={'sharey': False, 'sharex': True},
    height=5,
    aspect=1)

g.set_titles(col_template="{col_name}", size=16)
g.set_axis_labels("CHILDREN", "")
for ax in g.axes.flatten():
    ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
ff = ['bmi', 'age', 'children', 'premium']

df_melted = data.melt(id_vars=['expenses', 'gender'], value_vars=ff, var_name='variable', value_name='value')

g = sns.relplot(
    data=df_melted,
    x='expenses',
    y='value',
    hue='gender',
    col='variable',
    kind='scatter',
    palette=custom_palette2,
    facet_kws={'sharey': False, 'sharex': True},
    height=5,
    aspect=1)

g.set_titles(col_template="{col_name}", size=16)
g.set_axis_labels("EXPENSES", "")
for ax in g.axes.flatten():
    ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
ff = ['bmi', 'age', 'children', 'expenses']

df_melted = data.melt(id_vars=['premium', 'gender'], value_vars=ff, var_name='variable', value_name='value')

g = sns.relplot(
    data=df_melted,
    x='premium',
    y='value',
    hue='gender',
    col='variable',
    kind='scatter',
    palette=custom_palette2,
    facet_kws={'sharey': False, 'sharex': True},
    height=5,
    aspect=1)

g.set_titles(col_template="{col_name}", size=16)
g.set_axis_labels("PREMIUM", "")
for ax in g.axes.flatten():
    ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
sns.set(style="whitegrid")

columns = [
    'age', 'gender', 'bmi', 'children',
    'discount_eligibility', 'region', 'expenses', 'premium']

custom_palette = ["#B7D5D4", "#77878B", "#488286", "#305252", "#92B8A7", "#3B5A54"]

n = len(columns)
rows = (n + 1) // 2
fig, axes = plt.subplots(rows, 2, figsize=(18, rows * 6))

for i, col in enumerate(columns):
    ax = axes[i // 2, i % 2]
    if data[col].dtype in ['int64', 'float64']: 
        sns.kdeplot(
            data[col],
            fill=True,
            color=custom_palette[i % len(custom_palette)],
            alpha=0.6,
            linewidth=2,
            ax=ax
        )
        ax.set_xlabel(col, fontsize=14)
        ax.set_ylabel("Density", fontsize=14)
        ax.set_title(f'Distribution of {col}', fontsize=16)
    else: 
        sns.countplot(
            x=col,
            data=data,
            palette=custom_palette,
            ax=ax
        )
        ax.set_xlabel(col, fontsize=14)
        ax.set_ylabel("Count", fontsize=14)
        ax.set_title(f'Count Plot of {col}', fontsize=16)
        for p in ax.patches:
            height = p.get_height()
            ax.annotate(f'{height}', (p.get_x() + p.get_width() / 2, height),
                        ha='center', va='bottom', fontsize=11)

plt.tight_layout()
plt.show()

In [ ]:
numeric_cols = data.select_dtypes(include=['number']).columns

skew_values = data[numeric_cols].skew(axis=0, skipna=True)

print("Skewness of Numeric Columns:\n")
print(skew_values.sort_values(ascending=False).round(3))

In [ ]:
data_mean = data[numeric_cols].mean()

custom_palette = ["#B7D5D4", "#77878B", "#488286", "#305252", "#92B8A7", "#3B5A54"]

colors_to_use = (custom_palette * ((len(data_mean) // len(custom_palette)) + 1))[:len(data_mean)]

data_mean.plot(
    kind='barh',
    figsize=(15, 5),
    color=colors_to_use
)

plt.xlabel('Average')
plt.title("Average of Numerical Columns")
plt.tight_layout()
plt.show()

In [ ]:
data[numeric_cols].boxplot(figsize=(35,10),vert=False)

In [ ]:
numeric_cols = data.select_dtypes(include=['number']).columns

for i, column in enumerate(numeric_cols):
    plt.figure(figsize=(15, 3))
    sns.violinplot(
        x=data[column],
        palette=[custom_palette[i % len(custom_palette)]], 
        inner="quartile"
    )
    plt.title(f'Distribution of {column}', fontsize=18)
    plt.xlabel(column, fontsize=14)
    plt.ylabel('Density', fontsize=14)
    plt.grid(True)
    plt.show()

In [ ]:
for i, column in enumerate(ff):
    plt.figure(figsize=(15, 3))
    sns.boxplot(
        x=data[column],
        color=custom_palette[i % len(custom_palette)])
    
    plt.title(f'Distribution of {column}', fontsize=18)
    plt.xlabel(column, fontsize=14)

    stats = data[column].describe()
    stats_text = "\n".join([f"{key}: {value:.2f}" for key, value in stats.items()])
    plt.text(
        0.95, 0.95, stats_text,
        transform=plt.gca().transAxes,
        fontsize=12,
        verticalalignment='top',
        horizontalalignment='center',
        bbox=dict(boxstyle='round,pad=0.2', edgecolor='black', facecolor='lightgrey'))

    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
count_vars = [
    'gender',
    'children',
    'discount_eligibility',
    'region']
n_cols = 2
n_rows = (len(count_vars) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 10), dpi=90)
axes = axes.flatten()
for i, column in enumerate(count_vars):
    sns.countplot(
        x=column,
        data=data,
        palette=custom_palette,
        hue='gender',
        ax=axes[i]
    )
    axes[i].set_title(f'Count of {column}', fontsize=18)
    axes[i].set_xlabel(column, fontsize=14)
    axes[i].set_ylabel('Count', fontsize=14)
    axes[i].tick_params(axis='x', rotation=20, labelsize=12)
    axes[i].tick_params(axis='y', labelsize=12)
    axes[i].grid(True, linestyle='--', alpha=0.6)
    axes[i].legend(title='Gender', fontsize=11, title_fontsize=12)
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])
plt.tight_layout()
plt.show()

In [ ]:
metrics = [
    ('age', 'Age'),
    ('bmi', 'BMI'),
    ('children', 'Children'),
    ('expenses', 'Expenses'),
    ('premium', 'Premium')]

group_col = 'region' 

fig, axes = plt.subplots(3, 2, figsize=(25, 15))
fig.suptitle(f'Trends by {group_col.title()}', fontsize=30)

for i, (col, label) in enumerate(metrics):
    row, col_pos = divmod(i, 2)
    sns.lineplot(
        ax=axes[row, col_pos],
        x=group_col,
        y=col,
        data=data,
        ci=None,
        color=custom_palette[i % len(custom_palette)],
        linewidth=2
    )
    axes[row, col_pos].set_title(f'{label} by {group_col.title()}', fontsize=20)
    axes[row, col_pos].set_xlabel(group_col.title(), fontsize=16)
    axes[row, col_pos].set_ylabel(label, fontsize=16)
    axes[row, col_pos].tick_params(axis='x', rotation=45, labelsize=14)
    axes[row, col_pos].tick_params(axis='y', labelsize=14)
    axes[row, col_pos].grid(True, linestyle='--', alpha=0.7)
if len(metrics) < 6:
    fig.delaxes(axes[2, 1])  

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
custom_palette = ["#B7D5D4", "#77878B", "#488286", "#305252", "#92B8A7", "#3B5A54"]

fig, axes = plt.subplots(2, 1, figsize=(25, 15))

sns.stripplot(
    data=data,
    x='expenses',
    y='region',
    hue='children',
    palette=custom_palette,
    orient='h',
    ax=axes[0],
    dodge=True
)
axes[0].set_title('Expenses by Region and Children Count', fontsize=16)
axes[0].legend(loc='lower right', title='Children')

# Boxplot
sns.boxplot(
    data=data,
    x='expenses',
    y='region',
    palette=custom_palette,
    orient='h',
    ax=axes[1]
)
axes[1].set_title('Distribution of Expenses by Region', fontsize=16)

plt.tight_layout()
plt.show()

In [ ]:
def StDev_method(data, n, features):
    outliers = []
    total_outliers = 0
    
    for column in features:
        data_mean = data[column].mean()
        data_std = data[column].std()
        cut_off = data_std * 3

        high_outliers = data[data[column] > data_mean + cut_off].index
        low_outliers = data[data[column] < data_mean - cut_off].index

        outliers.extend(high_outliers)
        outliers.extend(low_outliers)

        print(f"{column}: {len(high_outliers) + len(low_outliers)} outliers")

        total_outliers += len(high_outliers) + len(low_outliers)

    outliers_counter = Counter(outliers)
    OUT = [idx for idx, count in outliers_counter.items() if count > n]

    print(f'\n🔎 Total unique outlier rows (appearing in > {n} features): {len(OUT)}')
    print(f'📌 Total individual outlier points detected: {total_outliers}')
    
    return OUT

Outliers_StDev = StDev_method(data, 1, ff)

data_out = data.drop(Outliers_StDev, axis=0).reset_index(drop=True)

In [ ]:
gender_dummies = pd.get_dummies(data_out['gender'], dtype=int, drop_first=True)
discount_eligibility_dummies = pd.get_dummies(data_out['discount_eligibility'], dtype=int, drop_first=True)
region_dummies = pd.get_dummies(data_out['region'], dtype=int, drop_first=False)

data_out = pd.concat([data_out.iloc[:, :-1], region_dummies, data_out.iloc[:, -1]], axis=1)
data_out.drop(columns=['region'], inplace=True)
data_out['gender'] = gender_dummies
data_out['discount_eligibility'] = discount_eligibility_dummies

data_out

In [ ]:
plt.figure(figsize=(20, 12))
sns.heatmap(
    data_out.corr(numeric_only=True),
    cmap=custom_cmap,
    annot=True,
    linewidths=0.6,
    cbar=False)

plt.xticks(rotation=60, size=10)
plt.yticks(size=10)
plt.title('Analysis of Correlations', size=20)
plt.tight_layout()
plt.show()

<a id="2"></a>

<h1 style="
    background-color:#488286; 
    background-size: cover;
    background-position: center;
    font-family: 'Times New Roman', serif;
    font-size: 1.5em;
    color: white;
    text-align: center;
    padding: 15px;
    border-radius: 15px 50px;
    border: 1px solid black;
">
    Modeling
</h1>

### <span style="color: #305252;">🔍 Regression Modeling Pipeline Overview</span>

📊 **1. Data Preprocessing**

* Applied `pd.get_dummies()` to convert categorical variables into numerical format.
* Used `StandardScaler` to normalize numerical features for consistent scaling.

🧠 **2. Regression Models (14 total), grouped as:**

* **Linear Models:** Ridge, Lasso, ElasticNet, BayesianRidge, HuberRegressor
* **Tree-Based Model:** DecisionTreeRegressor
* **Ensemble Trees:** RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
* **Nearest Neighbors:** KNeighborsRegressor
* **Kernel-Based:** Support Vector Regressor (SVR)
* **Advanced Gradient Boosting:** XGBoost, LightGBM, CatBoost

🔧 **3. Hyperparameter Tuning**

* Employed `GridSearchCV` with specified parameter grids for all models.
* Used 5-fold cross-validation (`cv=5`).
* Optimized based on negative mean squared error (`neg_mean_squared_error`).
* Enabled parallel processing with `n_jobs=-1` for efficiency.

📈 **4. Model Evaluation Metrics**

* Evaluated models on training and testing data with:

  * Mean Absolute Error (MAE)
  * Mean Squared Error (MSE)
  * Coefficient of Determination (R²)
  * Adjusted R²

📊 **5. Final Visualizations**

* MAE comparison: Train vs Test
* R² comparison: Train vs Test
* Histogram of Actual-to-Predicted value ratios
* Residual distribution across models
* Actual vs Predicted scatter plots for each model

## - Forecasting Expenses

In [ ]:
warnings.filterwarnings("ignore")

# Custom color palette for visualization
custom_palette = ['#B7D5D4', '#77878B', '#488286', '#305252']

# Select feature columns and target variable
features = ['age', 'gender', 'bmi', 'children', 'discount_eligibility',
            'northeast', 'northwest', 'southeast', 'southwest']
target = 'expenses' 

X = data_out[features]
y = data_out[target]

X = pd.get_dummies(X, columns=['gender', 'discount_eligibility'], drop_first=True)

# Split the dataset into training and testing subsets with random shuffling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and apply standard scaler to numerical features for normalization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Dictionary of candidate regression models with default hyperparameters
model_candidates = {
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet': ElasticNet(),
    'Bayesian Ridge': BayesianRidge(),
    'Huber Regressor': HuberRegressor(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'AdaBoost': AdaBoostRegressor(),
    'K-Nearest Neighbors': KNeighborsRegressor(),
    'Support Vector Regressor': SVR(),
    'XGBoost': XGBRegressor(verbosity=0), 
    'LightGBM': LGBMRegressor(verbose=-1),  
    'CatBoost': CatBoostRegressor(verbose=0) 
}

# Hyperparameter grids for GridSearch cross-validation
param_grids = {
    'Ridge Regression': {'alpha': [0.1, 1.0, 10.0]},
    'Lasso Regression': {'alpha': [0.1, 1.0, 10.0]},
    'ElasticNet': {'alpha': [0.1, 1.0, 10.0], 'l1_ratio': [0.1, 0.5, 0.9]},
    'Bayesian Ridge': {
        'alpha_1': [1e-6, 1e-5, 1e-4], 'alpha_2': [1e-6, 1e-5, 1e-4],
        'lambda_1': [1e-6, 1e-5, 1e-4], 'lambda_2': [1e-6, 1e-5, 1e-4]
    },
    'Huber Regressor': {'alpha': [0.0001, 0.001, 0.01]},
    'Decision Tree': {'max_depth': [None, 10, 20, 30]},
    'Random Forest': {'n_estimators': [100, 200, 500], 'max_depth': [None, 10, 20, 30]},
    'Gradient Boosting': {'n_estimators': [100, 200, 500], 'learning_rate': [0.01, 0.1, 0.2]},
    'AdaBoost': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2]},
    'K-Nearest Neighbors': {'n_neighbors': [3, 5, 10], 'weights': ['uniform', 'distance']},
    'Support Vector Regressor': {'kernel': ['linear', 'rbf'], 'C': [0.1, 1.0, 10.0], 'epsilon': [0.01, 0.1, 0.2]},
    'XGBoost': {'n_estimators': [100, 200, 500], 'learning_rate': [0.01, 0.1, 0.2]},
    'LightGBM': {'n_estimators': [100, 200, 500], 'learning_rate': [0.01, 0.1, 0.2]},
    'CatBoost': {'iterations': [100, 200, 500], 'learning_rate': [0.01, 0.1, 0.2]}
}

# Helper function to compute adjusted R²
def adjusted_r2(r2, n_samples, n_features):
    return 1 - (1 - r2) * (n_samples - 1) / (n_samples - n_features - 1)

# Initialize dictionary to store results and predictions
results = {
    'Model': [],
    'MSE_Train': [],
    'R2_Train': [],
    'Adj_R2_Train': [],
    'MAE_Train': [],
    'MSE_Test': [],
    'R2_Test': [],
    'Adj_R2_Test': [],
    'MAE_Test': [],
    'y_test_true': [],
    'y_test_pred': [],
    'y_train_true': [],
    'y_train_pred': []
}

# Train, tune, predict, and evaluate models
for name, model in model_candidates.items():
    print(f"Training and tuning {name}...")

    if name in param_grids:
        grid_search = GridSearchCV(model, param_grids[name], scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=0)
        grid_search.fit(X_train_scaled, y_train) 
        best_model = grid_search.best_estimator_
    else:
        best_model = model
        best_model.fit(X_train_scaled, y_train) 

    y_train_pred = best_model.predict(X_train_scaled)
    y_test_pred = best_model.predict(X_test_scaled)

    mse_train = mean_squared_error(y_train, y_train_pred)
    r2_train = r2_score(y_train, y_train_pred)
    adj_r2_train = adjusted_r2(r2_train, len(y_train), X_train_scaled.shape[1])
    mae_train = mean_absolute_error(y_train, y_train_pred)

    mse_test = mean_squared_error(y_test, y_test_pred)
    r2_test = r2_score(y_test, y_test_pred)
    adj_r2_test = adjusted_r2(r2_test, len(y_test), X_test_scaled.shape[1])
    mae_test = mean_absolute_error(y_test, y_test_pred)

    results['Model'].append(name)
    results['MSE_Train'].append(mse_train)
    results['R2_Train'].append(r2_train)
    results['Adj_R2_Train'].append(adj_r2_train)
    results['MAE_Train'].append(mae_train)
    results['MSE_Test'].append(mse_test)
    results['R2_Test'].append(r2_test)
    results['Adj_R2_Test'].append(adj_r2_test)
    results['MAE_Test'].append(mae_test)
    results['y_test_true'].append(y_test)
    results['y_test_pred'].append(y_test_pred)
    results['y_train_true'].append(y_train)
    results['y_train_pred'].append(y_train_pred)

# Convert to DataFrame and sort by test R2
results_df = pd.DataFrame(results).sort_values(by='R2_Test', ascending=False).reset_index(drop=True)

# --------- Plot MAE Train vs Test --------------
plt.figure(figsize=(12,6))
sns.scatterplot(x='MAE_Train', y='MAE_Test', data=results_df, hue='Model', palette=custom_palette, s=100)
plt.plot([results_df['MAE_Train'].min(), results_df['MAE_Train'].max()],
         [results_df['MAE_Train'].min(), results_df['MAE_Train'].max()],
         'r--', lw=2)
plt.title('Mean Absolute Error (Train vs Test)', fontsize=16)
plt.xlabel('MAE Train')
plt.ylabel('MAE Test')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()

# --------- Plot R2 Train vs Test --------------
plt.figure(figsize=(12,6))
sns.scatterplot(x='R2_Train', y='R2_Test', data=results_df, hue='Model', palette=custom_palette, s=100)
plt.plot([results_df['R2_Train'].min(), results_df['R2_Train'].max()],
         [results_df['R2_Train'].min(), results_df['R2_Train'].max()],
         'r--', lw=2)
plt.title('R² Score (Train vs Test)', fontsize=16)
plt.xlabel('R² Train')
plt.ylabel('R² Test')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()

# --------- Scatter plot of Actual/Predicted Ratio (Test data) --------------
plt.figure(figsize=(14,8))
ratios = []
models_expanded = []
for i, row in results_df.iterrows():
    ratio = row['y_test_true'] / (row['y_test_pred'] + 1e-8)  # Avoid div by zero
    ratios.extend(ratio)
    models_expanded.extend([row['Model']]*len(ratio))

df_ratio = pd.DataFrame({'Ratio': ratios, 'Model': models_expanded})
sns.histplot(data=df_ratio, x='Ratio', hue='Model', element='step', stat='count', palette=custom_palette, bins=30)
plt.title('Distribution of Actual/Predicted Ratio by Model (Test Data)', fontsize=16)
plt.xlabel('Actual / Predicted')
plt.ylabel('Count')
plt.grid(True)
plt.show()

# --------- Residuals distribution plot (Test data) --------------
plt.figure(figsize=(14,8))
for i, row in results_df.iterrows():
    residuals = row['y_test_true'] - row['y_test_pred']
    sns.kdeplot(residuals, label=row['Model'], fill=True, alpha=0.3)
plt.title('Residuals Distribution (Test Data)', fontsize=16)
plt.xlabel('Residuals (Actual - Predicted)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()

# --------- Actual vs Predicted plot for all models --------------
fig, axs = plt.subplots(len(results_df), 1, figsize=(10, len(results_df)*4), sharex=True)
if len(results_df) == 1:
    axs = [axs]

for i, row in results_df.iterrows():
    sns.scatterplot(x=row['y_test_true'], y=row['y_test_pred'], ax=axs[i], color=custom_palette[i % len(custom_palette)], alpha=0.6)
    axs[i].plot([row['y_test_true'].min(), row['y_test_true'].max()], [row['y_test_true'].min(), row['y_test_true'].max()], 'r--')
    axs[i].set_title(f'Actual vs Predicted Premium - {row["Model"]}')
    axs[i].set_xlabel('Actual Premium')
    axs[i].set_ylabel('Predicted Premium')
    axs[i].grid(True)

plt.tight_layout()
plt.show()

# Custom palette
custom_palette = ['#B7D5D4', '#77878B', '#488286', '#305252']
custom_cmap = LinearSegmentedColormap.from_list("custom", custom_palette)

# --------- Styled Performance Summary Table ---------
styled_results = results_df[['Model', 'MSE_Train', 'MAE_Train', 'R2_Train', 'Adj_R2_Train',
                             'MSE_Test', 'MAE_Test', 'R2_Test', 'Adj_R2_Test']].style\
    .background_gradient(cmap=custom_cmap, axis=1)\
    .format({
        'MSE_Train': "{:.2f}",
        'MAE_Train': "{:.2f}",
        'R2_Train': "{:.3f}",
        'Adj_R2_Train': "{:.3f}",
        'MSE_Test': "{:.2f}",
        'MAE_Test': "{:.2f}",
        'R2_Test': "{:.3f}",
        'Adj_R2_Test': "{:.3f}",
    })\
    .set_caption("Comprehensive Regression Models Performance Summary - Target: Expenses")

display(styled_results)


# Additional line plot for train vs test R2 scores
plt.figure(figsize=(14, 7))
df_long = pd.melt(results_df[['Model', 'R2_Train', 'R2_Test']], id_vars='Model', 
                  value_vars=['R2_Train', 'R2_Test'],
                  var_name='Dataset', value_name='R2_Score')

palette = ['#488286', '#B7D5D4'] 

sns.lineplot(data=df_long, x='Model', y='R2_Score', hue='Dataset', marker='o', palette=palette)
plt.xticks(rotation=45, fontsize=12)
plt.title('R² Score Comparison: Train vs Test for All Models')
plt.xlabel('Model')
plt.ylabel('R² Score')
plt.legend(title='Dataset')
plt.grid(True)
plt.tight_layout()
plt.show()

# Additional line plot for MAE comparison
plt.figure(figsize=(14, 6))
plt.plot(results_df['Model'], results_df['MAE_Train'], marker='o', label='Train MAE', color='#B7D5D4')
plt.plot(results_df['Model'], results_df['MAE_Test'], marker='o', label='Test MAE', color='#305252')
plt.xticks(rotation=45, ha='right')
plt.title('MAE Comparison Across Models')
plt.xlabel('Models')
plt.ylabel('Mean Absolute Error')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

<span style="font-weight:bold; font-size:1.3em;">✅ 1. Top Performing Models</span>

| Model               | R² Test   | MAE Test | Overfitting Assessment                                                 |
| ------------------- | --------- | -------- | ---------------------------------------------------------------------- |
| ✅ Gradient Boosting | **0.901** | 2494.13  | ❌ Minimal overfitting; train and test R² are closely aligned          |
| ✅ XGBoost           | **0.898** | 2393.28  | ⚠️ Slight overfitting; excellent train R² (0.932) with small test drop |
| ✅ LightGBM          | **0.897** | 2462.08  | ⚠️ Slight overfitting; solid train R² (0.910) and stable test accuracy  |
| ✅ CatBoost          | **0.896** | 2564.63  | ⚠️ Slight overfitting; comparable train/test R², slightly higher MAE   |
| ✅ AdaBoost          | **0.892** | 2936.20  | ⚠️ Mild overfitting; higher MAE than other boosting models             |
| ✅ Random Forest     | **0.883** | 2591.72  | ⚠️ Noticeable overfitting; very high train R² (0.966) vs test R²        |

**Key Insights:**

* **Gradient Boosting** delivers best test R² with balanced low MAE and minimal overfitting.
* **XGBoost** and **LightGBM** provide strong accuracy with slightly better train fits.
* **CatBoost** remains competitive, especially in handling categorical data.
* **AdaBoost** shows stable performance but with relatively higher errors.
* **Random Forest** has excellent train performance but suffers from overfitting.

---

<span style="font-weight:bold; font-size:1.3em;">⚠️ 2. Highly Overfitted Models</span>

| Model                 | R² Train | R² Test | Overfitting Severity                                                   |
| --------------------- | -------- | ------- | --------------------------------------------------------------------- |
| ❌ K-Nearest Neighbors | 1.000    | 0.819   | ✅ Severe overfitting; perfect train fit, large drop on test          |
| ❌ Decision Tree       | 0.984    | 0.789   | ✅ Significant overfitting; poor generalization despite low train error|

**Summary:**

* These models drastically overfit training data but fail to generalize well.
* Use with pruning, regularization, or ensemble techniques to improve.

---

<span style="font-weight:bold; font-size:1.3em;">⛔ 3. Underperforming Models</span>

| Model                          | R² Test | MAE Test |
| ------------------------------ | ------- | -------- |
| Support Vector Regressor (SVR) | 0.467   | 4871.32  |
| Huber Regressor                | 0.791   | 3355.37  |
| Lasso, Ridge, ElasticNet       | ~0.805  | ~4200    |
| Bayesian Ridge                 | 0.806   | 4184.80  |

**Observations:**

* These models struggle with the dataset’s nonlinear and complex patterns.
* High error and low R² indicate limited suitability for premium prediction.

---

<span style="font-weight:bold; font-size:1.3em;">⚡ 4. Special Case: AdaBoost</span>

| Model    | R² Train | R² Test | MAE Test |
| -------- | -------- | ------- | -------- |
| AdaBoost | 0.848    | 0.892   | 2936.20  |

* AdaBoost shows consistent behavior with relatively small train-test R² gap.
* Slightly weaker than other boosting methods in accuracy and MAE.

---

<span style="font-weight:bold; font-size:1.3em;">💡 Final Recommendations</span>

**Top recommended models:**

* `Gradient Boosting`: Best trade-off between accuracy and generalization.
* `XGBoost` and `LightGBM`: Highly effective and fast alternatives.
* `CatBoost`: Strong option for categorical feature handling.

**Models to avoid:**

* `K-Nearest Neighbors` and `Decision Tree`: Overfitting issues, poor test performance.
* `Support Vector Regressor`, `Huber`, and linear models: Low accuracy and high errors.



In [ ]:
warnings.filterwarnings("ignore")

best_models = {} 

for name, model in model_candidates.items():
    print(f"Training and tuning {name}...")
    if name in param_grids:
        grid_search = GridSearchCV(model, param_grids[name], scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=0)
        grid_search.fit(X_train_scaled, y_train)
        best_model = grid_search.best_estimator_
    else:
        best_model = model
        best_model.fit(X_train_scaled, y_train)
    
    best_models[name] = best_model
    
# Example input 
input_data = {
    'age': 28,
    'gender': 1,  # 1 for male, 0 for female
    'bmi': 33.0,
    'children': 3,
    'discount_eligibility': 0,
    'northeast': 0,
    'northwest': 0,
    'southeast': 1,
    'southwest': 0,
}

# Convert to DataFrame
input_df = pd.DataFrame([input_data])

# One-hot encode categorical columns, as done in training
input_df = pd.get_dummies(input_df, columns=['gender', 'discount_eligibility'], drop_first=True)

# Ensure all expected columns exist (add missing dummy columns if any)
expected_cols = X.columns.tolist()  # X is our original training features DataFrame
for col in expected_cols:
    if col not in input_df.columns:
        input_df[col] = 0

# Reorder columns to match training data
input_df = input_df[expected_cols]

# Scale input features using the same scaler as training
input_scaled = scaler.transform(input_df)

# Predict with each best model (change this list as needed)
for model_name in ['Gradient Boosting', 'XGBoost', 'LightGBM', 'AdaBoost', 'CatBoost']:
    model = best_models[model_name]  # Assuming we saved best models after grid search
    pred = model.predict(input_scaled)
    print(f'\033[96m {model_name} prediction: {pred[0]:.2f}')

## - Forecasting Premiums

In [ ]:
warnings.filterwarnings("ignore")

# Custom color palette for visualization
custom_palette = ['#B7D5D4', '#77878B', '#488286', '#305252']

# Select feature columns and target variable (premium as target)
features = ['age', 'gender', 'bmi', 'children', 'discount_eligibility',
            'northeast', 'northwest', 'southeast', 'southwest']
target = 'premium' 

X = data_out[features]
y = data_out[target]

# One-hot encoding categorical columns
X = pd.get_dummies(X, columns=['gender', 'discount_eligibility'], drop_first=True)

# Split dataset into training and testing subsets with random shuffling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Candidate regression models with default hyperparameters
model_candidates = {
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet': ElasticNet(),
    'Bayesian Ridge': BayesianRidge(),
    'Huber Regressor': HuberRegressor(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'AdaBoost': AdaBoostRegressor(),
    'K-Nearest Neighbors': KNeighborsRegressor(),
    'Support Vector Regressor': SVR(),
    'XGBoost': XGBRegressor(verbosity=0), 
    'LightGBM': LGBMRegressor(verbose=-1),  
    'CatBoost': CatBoostRegressor(verbose=0)
}

# Hyperparameter grids for GridSearchCV
param_grids = {
    'Ridge Regression': {'alpha': [0.1, 1.0, 10.0]},
    'Lasso Regression': {'alpha': [0.1, 1.0, 10.0]},
    'ElasticNet': {'alpha': [0.1, 1.0, 10.0], 'l1_ratio': [0.1, 0.5, 0.9]},
    'Bayesian Ridge': {
        'alpha_1': [1e-6, 1e-5, 1e-4], 'alpha_2': [1e-6, 1e-5, 1e-4],
        'lambda_1': [1e-6, 1e-5, 1e-4], 'lambda_2': [1e-6, 1e-5, 1e-4]
    },
    'Huber Regressor': {'alpha': [0.0001, 0.001, 0.01]},
    'Decision Tree': {'max_depth': [None, 10, 20, 30]},
    'Random Forest': {'n_estimators': [100, 200, 500], 'max_depth': [None, 10, 20, 30]},
    'Gradient Boosting': {'n_estimators': [100, 200, 500], 'learning_rate': [0.01, 0.1, 0.2]},
    'AdaBoost': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2]},
    'K-Nearest Neighbors': {'n_neighbors': [3, 5, 10], 'weights': ['uniform', 'distance']},
    'Support Vector Regressor': {'kernel': ['linear', 'rbf'], 'C': [0.1, 1.0, 10.0], 'epsilon': [0.01, 0.1, 0.2]},
    'XGBoost': {'n_estimators': [100, 200, 500], 'learning_rate': [0.01, 0.1, 0.2]},
    'LightGBM': {'n_estimators': [100, 200, 500], 'learning_rate': [0.01, 0.1, 0.2]},
    'CatBoost': {'iterations': [100, 200, 500], 'learning_rate': [0.01, 0.1, 0.2]}
}

# Adjusted R² calculation helper
def adjusted_r2(r2, n_samples, n_features):
    return 1 - (1 - r2) * (n_samples - 1) / (n_samples - n_features - 1)

# Initialize storage for results
results = {
    'Model': [],
    'MSE_Train': [],
    'R2_Train': [],
    'Adj_R2_Train': [],
    'MAE_Train': [],
    'MSE_Test': [],
    'R2_Test': [],
    'Adj_R2_Test': [],
    'MAE_Test': [],
    'y_test_true': [],
    'y_test_pred': [],
    'y_train_true': [],
    'y_train_pred': []
}

# Train, tune, predict, and evaluate each model
for name, model in model_candidates.items():
    print(f"Training and tuning {name}...")
    if name in param_grids:
        grid_search = GridSearchCV(model, param_grids[name], scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=0)
        grid_search.fit(X_train_scaled, y_train)
        best_model = grid_search.best_estimator_
    else:
        best_model = model
        best_model.fit(X_train_scaled, y_train)

    y_train_pred = best_model.predict(X_train_scaled)
    y_test_pred = best_model.predict(X_test_scaled)

    mse_train = mean_squared_error(y_train, y_train_pred)
    r2_train = r2_score(y_train, y_train_pred)
    adj_r2_train = adjusted_r2(r2_train, len(y_train), X_train_scaled.shape[1])
    mae_train = mean_absolute_error(y_train, y_train_pred)

    mse_test = mean_squared_error(y_test, y_test_pred)
    r2_test = r2_score(y_test, y_test_pred)
    adj_r2_test = adjusted_r2(r2_test, len(y_test), X_test_scaled.shape[1])
    mae_test = mean_absolute_error(y_test, y_test_pred)

    results['Model'].append(name)
    results['MSE_Train'].append(mse_train)
    results['R2_Train'].append(r2_train)
    results['Adj_R2_Train'].append(adj_r2_train)
    results['MAE_Train'].append(mae_train)
    results['MSE_Test'].append(mse_test)
    results['R2_Test'].append(r2_test)
    results['Adj_R2_Test'].append(adj_r2_test)
    results['MAE_Test'].append(mae_test)
    results['y_test_true'].append(y_test)
    results['y_test_pred'].append(y_test_pred)
    results['y_train_true'].append(y_train)
    results['y_train_pred'].append(y_train_pred)

# Convert results to DataFrame and sort by test R²
results_df = pd.DataFrame(results).sort_values(by='R2_Test', ascending=False).reset_index(drop=True)

# Visualizations
plt.figure(figsize=(12,6))
sns.scatterplot(x='MAE_Train', y='MAE_Test', data=results_df, hue='Model', palette=custom_palette, s=100)
plt.plot([results_df['MAE_Train'].min(), results_df['MAE_Train'].max()],
         [results_df['MAE_Train'].min(), results_df['MAE_Train'].max()],
         'r--', lw=2)
plt.title('Mean Absolute Error (Train vs Test)')
plt.xlabel('MAE Train')
plt.ylabel('MAE Test')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()

plt.figure(figsize=(12,6))
sns.scatterplot(x='R2_Train', y='R2_Test', data=results_df, hue='Model', palette=custom_palette, s=100)
plt.plot([results_df['R2_Train'].min(), results_df['R2_Train'].max()],
         [results_df['R2_Train'].min(), results_df['R2_Train'].max()],
         'r--', lw=2)
plt.title('R² Score (Train vs Test)')
plt.xlabel('R² Train')
plt.ylabel('R² Test')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()

plt.figure(figsize=(14,8))
ratios = []
models_expanded = []
for i, row in results_df.iterrows():
    ratio = row['y_test_true'] / (row['y_test_pred'] + 1e-8)  # prevent division by zero
    ratios.extend(ratio)
    models_expanded.extend([row['Model']]*len(ratio))

df_ratio = pd.DataFrame({'Ratio': ratios, 'Model': models_expanded})
sns.histplot(data=df_ratio, x='Ratio', hue='Model', element='step', stat='count', palette=custom_palette, bins=30)
plt.title('Distribution of Actual/Predicted Ratio by Model (Test Data)')
plt.xlabel('Actual / Predicted')
plt.ylabel('Count')
plt.grid(True)
plt.show()

plt.figure(figsize=(14,8))
for i, row in results_df.iterrows():
    residuals = row['y_test_true'] - row['y_test_pred']
    sns.kdeplot(residuals, label=row['Model'], fill=True, alpha=0.3)
plt.title('Residuals Distribution (Test Data)')
plt.xlabel('Residuals (Actual - Predicted)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()

fig, axs = plt.subplots(len(results_df), 1, figsize=(10, len(results_df)*4), sharex=True)
if len(results_df) == 1:
    axs = [axs]

for i, row in results_df.iterrows():
    sns.scatterplot(x=row['y_test_true'], y=row['y_test_pred'], ax=axs[i], color=custom_palette[i % len(custom_palette)], alpha=0.6)
    axs[i].plot([row['y_test_true'].min(), row['y_test_true'].max()], [row['y_test_true'].min(), row['y_test_true'].max()], 'r--')
    axs[i].set_title(f'Actual vs Predicted Premium - {row["Model"]}')
    axs[i].set_xlabel('Actual Premium')
    axs[i].set_ylabel('Predicted Premium')
    axs[i].grid(True)

plt.tight_layout()
plt.show()

# Color map for table styling
custom_cmap = LinearSegmentedColormap.from_list("custom", custom_palette)

# Styled summary table
styled_results = results_df[['Model', 'MSE_Train', 'MAE_Train', 'R2_Train', 'Adj_R2_Train',
                             'MSE_Test', 'MAE_Test', 'R2_Test', 'Adj_R2_Test']].style\
    .background_gradient(cmap=custom_cmap, axis=1)\
    .format({
        'MSE_Train': "{:.2f}",
        'MAE_Train': "{:.2f}",
        'R2_Train': "{:.3f}",
        'Adj_R2_Train': "{:.3f}",
        'MSE_Test': "{:.2f}",
        'MAE_Test': "{:.2f}",
        'R2_Test': "{:.3f}",
        'Adj_R2_Test': "{:.3f}",
    })\
    .set_caption("Comprehensive Regression Models Performance Summary - Target: Premium")

display(styled_results)

# Additional line plot for train vs test R2 scores
plt.figure(figsize=(14, 7))
df_long = pd.melt(results_df[['Model', 'R2_Train', 'R2_Test']], id_vars='Model', 
                  value_vars=['R2_Train', 'R2_Test'],
                  var_name='Dataset', value_name='R2_Score')

palette = ['#488286', '#B7D5D4'] 

sns.lineplot(data=df_long, x='Model', y='R2_Score', hue='Dataset', marker='o', palette=palette)
plt.xticks(rotation=45, fontsize=12)
plt.title('R² Score Comparison: Train vs Test for All Models')
plt.xlabel('Model')
plt.ylabel('R² Score')
plt.legend(title='Dataset')
plt.grid(True)
plt.tight_layout()
plt.show()

# Additional line plot for MAE comparison
plt.figure(figsize=(14, 6))
plt.plot(results_df['Model'], results_df['MAE_Train'], marker='o', label='Train MAE', color='#B7D5D4')
plt.plot(results_df['Model'], results_df['MAE_Test'], marker='o', label='Test MAE', color='#305252')
plt.xticks(rotation=45, ha='right')
plt.title('MAE Comparison Across Models')
plt.xlabel('Models')
plt.ylabel('Mean Absolute Error')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

<span style="font-weight:bold; font-size:1.3em;">✅ 1. Top Performing Models</span>

| Model               | R² Test   | MAE Test | Overfitting Assessment                                              |
| ------------------- | --------- | -------- | ------------------------------------------------------------------- |
| ✅ Random Forest     | **0.930** | 44.22    | ❌ Minimal overfitting; very close train-test R² values             |
| ✅ Gradient Boosting | **0.931** | 51.09    | ❌ Minimal overfitting; consistent generalization                   |
| ✅ XGBoost           | **0.928** | 44.21    | ⚠️ Slight overfitting; small drop from train to test                |
| ✅ CatBoost          | **0.925** | 49.26    | ⚠️ Slight overfitting; steady performance with elevated error rates |
| ✅ LightGBM          | **0.913** | 57.17    | ⚠️ Mild overfitting; relatively higher test error                   |
| ✅ AdaBoost          | **0.892** | 82.89    | ⚠️ Moderate overfitting; highest MAE among top models               |

**Key Insights:**

* **Random Forest** and **Gradient Boosting** show top test R² with RF having lower MAE, making it a strong overall choice.
* **XGBoost** and **CatBoost** perform similarly with slightly more overfitting.
* **LightGBM** performs well but might improve with further tuning.
* **AdaBoost** lags behind in precision, showing moderate overfitting.

---

<span style="font-weight:bold; font-size:1.3em;">⚠️ 2. Highly Overfitted Models</span>

| Model                 | R² Train | R² Test | Overfitting Severity                                              |
| --------------------- | -------- | ------- | ----------------------------------------------------------------- |
| ❌ Decision Tree       | 0.983    | 0.890   | ✅ Noticeable overfitting; large train-test performance gap       |
| ❌ K-Nearest Neighbors | 1.000    | 0.735   | ✅ Severe overfitting; perfect train fit, poor generalization     |

**Summary:**

* **Decision Tree** and **KNN** models severely overfit, excelling on train but failing on unseen data.
* Pruning or regularization needed; ensemble alternatives recommended.

---

<span style="font-weight:bold; font-size:1.3em;">⛔ 3. Underperforming Models</span>

| Model                          | R² Test | MAE Test |
| ------------------------------ | ------- | -------- |
| Support Vector Regressor (SVR) | 0.488   | 112.42   |
| Huber Regressor                | 0.504   | 111.50   |
| Ridge, Lasso, ElasticNet       | ~0.69   | ~120     |
| Bayesian Ridge                 | ~0.69   | ~120     |

**Observations:**

* Linear and simpler models like **SVR**, **Huber**, and Ridge-family regressors struggle with the dataset’s nonlinear complexity.
* High errors and low R² make them less reliable for premium prediction.

---

<span style="font-weight:bold; font-size:1.3em;">⚡ 4. Special Case: AdaBoost</span>

| Model    | R² Train | R² Test | MAE Test |
| -------- | -------- | ------- | -------- |
| AdaBoost | 0.859    | 0.892   | 82.89    |

* AdaBoost shows moderate overfitting and comparatively weaker performance.
* Could be used cautiously with fine tuning or in ensemble stacking.

---

<span style="font-weight:bold; font-size:1.3em;">💡 Final Recommendations</span>

**Top recommended models:**

* `Random Forest`: Best balance of accuracy and generalization.
* `Gradient Boosting`, `XGBoost`, `CatBoost`, `LightGBM`: Strong contenders, worth tuning.
* `AdaBoost`: Backup option with attention to hyperparameters.

**Models to avoid:**

* `Decision Tree`, `K-Nearest Neighbors`: Prone to overfitting; poor test results.
* `SVR`, `Huber`, and linear models: Insufficient modeling capacity for data complexity.

In [ ]:
warnings.filterwarnings("ignore")

# Select only the best performing models from your results
best_model_names = ['Random Forest', 'Gradient Boosting', 'XGBoost', 'CatBoost', 'LightGBM']

best_models = {}

for name in best_model_names:
    model = model_candidates[name]
    print(f"Training and tuning {name}...")
    if name in param_grids:
        grid_search = GridSearchCV(model, param_grids[name], scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=0)
        grid_search.fit(X_train_scaled, y_train)
        best_model = grid_search.best_estimator_
    else:
        best_model = model
        best_model.fit(X_train_scaled, y_train)
    
    best_models[name] = best_model

# Example input for prediction
input_data = {
    'age': 28,
    'gender': 1,  # 1 for male, 0 for female
    'bmi': 33.0,
    'children': 3,
    'discount_eligibility': 0,
    'northeast': 0,
    'northwest': 0,
    'southeast': 1,
    'southwest': 0,
}

# Convert input to DataFrame
input_df = pd.DataFrame([input_data])

# One-hot encode categorical columns (same as training)
input_df = pd.get_dummies(input_df, columns=['gender', 'discount_eligibility'], drop_first=True)

# Add any missing columns to match training features
expected_cols = X.columns.tolist()
for col in expected_cols:
    if col not in input_df.columns:
        input_df[col] = 0

# Reorder columns to match training set
input_df = input_df[expected_cols]

# Scale input features
input_scaled = scaler.transform(input_df)

# Predict using the best models
for model_name in best_model_names:
    model = best_models[model_name]
    pred = model.predict(input_scaled)
    print(f'\033[96m{model_name} prediction: {pred[0]:.2f}')